In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import pandas as pd
import numpy as np

from cuml.ensemble import RandomForestClassifier
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler

In [ ]:
x_train=pd.read_csv("../input/wids2021training/x_train.csv").drop(columns=["Unnamed: 0"])
y_train=pd.read_csv("../input/wids2021training/y_train.csv").drop(columns=["Unnamed: 0"])
x_valid=pd.read_csv("../input/wids2021training/x_valid.csv").drop(columns=["Unnamed: 0"])
y_valid=pd.read_csv("../input/wids2021training/y_valid.csv").drop(columns=["Unnamed: 0"])

In [ ]:
for c in x_train.columns:
    x_train[c] = x_train[c].astype(np.float32)

In [ ]:
for c in y_train.columns:
    y_train[c] = y_train[c].astype(np.float32)

In [ ]:
for c in x_valid.columns:
    x_valid[c] = x_valid[c].astype(np.float32)

In [ ]:
for c in y_valid.columns:
    y_valid[c] = y_valid[c].astype(np.float32)

In [ ]:
# save best model
model_best = None
accuracy_best = 0
n_estimators_best = -1
max_depth_best = -1

In [ ]:
# objective function to maximize

In [ ]:
# trainable API
def train_rf(config):
    global accuracy_best
    global n_estimators_best
    global max_depth_best
    # TODO: train this
    n_estimators = int(config["n_estimators"])
    max_depth = int(config["max_depth"])
    model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth)
    model_trained = model.fit(x_train,y_train)
    valid_accuracy = model_trained.score(x_valid,y_valid)
    #tune.report(mean_accuracy=valid_accuracy)
    if valid_accuracy > accuracy_best:
        model_best = model_trained
        accuracy_best = valid_accuracy
        n_estimators_best = n_estimators
        max_depth_best = max_depth
        print("best accuracy: {}".format(accuracy_best))
        print("best n estimators: {}".format(n_estimators_best))
        print("best max depth: {}".format(max_depth_best))

In [ ]:
pip uninstall -y dataclasses

In [ ]:
# scheduler
ahbs_scheduler = AsyncHyperBandScheduler(time_attr="training_iteration", metric="test_accuracy", mode="max", max_t=50, grace_period=1, reduction_factor=3, brackets=3)

In [ ]:
# search algorithm
n_estimators_range = (100, 1300)
max_depth_range = (2, 25)
search_space = {
    "n_estimators": tune.randint(n_estimators_range[0], n_estimators_range[1]),
    "max_depth": tune.randint(max_depth_range[0], max_depth_range[1]),
}

bayesopt_search = BayesOptSearch(space={'n_estimators': n_estimators_range, 'max_depth': max_depth_range},metric="test_accuracy",mode="max",utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})

# run the analysis
analysis = tune.run(train_rf, search_alg=bayesopt_search, resources_per_trial={"cpu": 2, "gpu": 1}, num_samples=50, scheduler=ahbs_scheduler)

In [ ]:
dfs = analysis.trial_dataframes
[d.mean_accuracy.plot() for d in dfs.values()]